In [32]:
import json
from pathlib import Path
from slugify import slugify
import shutil
import requests
import yaml
from datetime import datetime, date, time
from bs4 import BeautifulSoup
import html2text
import locale
locale.setlocale(locale.LC_TIME, "de_DE")

'de_DE'

In [2]:
def g(name, show=False):
    with open("db1033256-1.json", "r") as f:
        d = json.load(f)
    for index, table in enumerate(d):        
        try:        
            if show:
                print(index, table["name"])
            if table["name"] == name:
                return table["data"]
        except:
            pass

In [3]:
g("", show=True)

1 db1033256-1
2 ArticlePage
3 ArticlePage_Live
4 ArticlePage_Versions
5 Astropic
6 Astropic_Nachbearbeitung
7 Astrovid
8 Astrovid_Nachbearbeitung
9 Blog
10 BlogCategory
11 BlogEntry
12 BlogEntry_Live
13 BlogEntry_versions
14 BlogHolder
15 BlogHolder_Live
16 BlogHolder_versions
17 BlogPost
18 BlogPost_Authors
19 BlogPost_Categories
20 BlogPost_Live
21 BlogPost_Tags
22 BlogPost_Versions
23 BlogTag
24 BlogTree
25 BlogTree_Live
26 BlogTree_versions
27 Blog_Contributors
28 Blog_Editors
29 Blog_Live
30 Blog_Versions
31 Blog_Writers
32 CarouselSlide
33 CarouselSlide_Live
34 CarouselSlide_Versions
35 ChangeSet
36 ChangeSetItem
37 ChangeSetItem_ReferencedBy
38 ContactData
39 DownloadObjekt
40 ErrorPage
41 ErrorPage_Live
42 ErrorPage_Versions
43 File
44 FileAttachmentFieldTrack
45 FileLink
46 File_EditorGroups
47 File_Live
48 File_Versions
49 File_ViewerGroups
50 GalleryCategory
51 Group
52 Group_Members
53 Group_Roles
54 HomePage
55 HomePage_Live
56 HomePage_Versions
57 ListenObjekt
58 LoginAtt

In [4]:
g("Astropic")[0]

{'ID': '2',
 'ClassName': 'Astropic',
 'LastEdited': '2018-12-31 14:41:23',
 'Created': '2016-06-05 12:53:51',
 'Title': 'Sternhimmel - Bosenberg',
 'Autor': 'Mehmet Ergün',
 'Kamera': 'Nikon D750 DSLR',
 'ISO': '3200',
 'Blende': '2,8',
 'Belichtung': '20',
 'AufnahmeOrt': 'Bosenberg Bosenheim',
 'AufnahmeDatum': '2016-03-03',
 'Teleskop': None,
 'DatenZumObjekt': 'Sternhimmel über Bad Kreuznach (Ortsteil Bosenheim)',
 'BildID': '285',
 'AufnahmeZeit': '00:00:00',
 'Objektiv': 'Tamron 15-30mm',
 'Nachbearbeitung': 'Adobe Lightroom',
 'TeleskopExtras': None,
 'KameraFilter': None,
 'Frames': None,
 'CategoryID': '2',
 'CreatedByID': '9',
 'EditedByID': '9'}

In [5]:
def get_download_link(astropic):
    image_object = find_image(astropic)
    image_file_location = "http://sternwarte-kreuznach.de/assets/" + image_object["FileFilename"]
    return image_file_location

In [6]:
def download_file(file_location, file_destination):
    if not Path(file_destination).is_file():
        r = requests.get(file_location, stream=True)
        if r.status_code == 200:
            r.raw.decode_content = True
            with open(file_destination, 'wb') as f:
                shutil.copyfileobj(r.raw, f)
        else:
            print('Image could not be retreived!')    

In [7]:
def find_image(astropic):
    image_id = int(astropic["BildID"])
    file_database = g("File")
    result = [entry for entry in file_database if int(entry["ID"]) == image_id]
    assert len(result) == 1
    return result[0]

In [20]:
def get_category(num):
    res = [x["Title"] for x in g("GalleryCategory") if x["ID"] == str(num)]
    assert len(res) == 1, ValueError("Not exactly one match!, Expected result for {}, got {} results. Options: {}".format(num, len(res), ["{}: {}".format(x["ID"], x["Title"]) for x in g("GalleryCategory")]))
    return res[0]  

In [9]:
def write_file(folder, yaml_dict, content):
    with open(folder.joinpath("index.md"), "w") as md:
        markdown_output = [
            "---\n",
            yaml.dump(yaml_dict, allow_unicode=True),
            "---\n",
            content
        ]
        md.writelines(markdown_output)

In [38]:

translation_dict = {
    "KameraFilter": "Kamera Zubehör",
    "Frames": "Anzahl der Einzelbilder",
    "TeleskopExtras": "Teleskop Zubehör",
    "AufnahmeOrt": "Aufnahmeort",
    "CategoryID": "Kategorie"
}

def make_content(astropic):    
    yml_dict = dict()
    content = ""
    
    if astropic["DatenZumObjekt"]:
        content += "{}\n\n".format(astropic["DatenZumObjekt"])
    
    if astropic["AufnahmeZeit"] or astropic["AufnahmeDatum"]:
        content += "Aufgenommen"
        if astropic["AufnahmeDatum"]:            
            content += " am {}".format(datetime.strptime(astropic["AufnahmeDatum"], '%Y-%m-%d').date().strftime("%d. %B %Y"))
        if astropic["AufnahmeZeit"]:
            content += " um {} Uhr".format(datetime.strptime(astropic["AufnahmeZeit"], '%H:%M:%S').strftime('%H:%M'))
        content += ".\n\n"
            
    content += "| Daten zur Aufnahme | |\n"
    content += "| - | - |\n"
    for key, value in astropic.items():        
        if key in ["ID", "ClassName", "LastEdited", "Created", "Title", "BildID", "DatenZumObjekt", "EditedByID", "CreatedByID", "Autor", "AufnahmeDatum", "AufnahmeZeit"]:
            continue
        else:
            if astropic[key]:
                if key == "CategoryID":
                    try:
                        value = get_category(value)
                    except AssertionError:
                        continue
                if key == "Blende":
                    value = "f/{}".format(value)
                if key == "Belichtung":
                    if int(value) <= 0:
                        continue       
                    value = "{} sec.".format(value)
                if key == "Frames":
                    try:
                        if int(value) <= 1:
                            continue               
                    except ValueError:
                        pass
                if key in translation_dict:
                    key = translation_dict[key]    
                content += "| {} | {} |\n".format(key, value)
        
    assert astropic["Autor"]
    yml_dict["creator"] = astropic["Autor"]
    
    assert astropic["Title"]
    yml_dict["title"] = astropic["Title"]
    
    assert astropic["Created"]
    yml_dict["date"] = str(datetime.strptime(astropic["Created"], '%Y-%m-%d %H:%M:%S'))
    
    
    return content, yml_dict

print(make_content(g("Astropic")[9])[0])

Aufgenommen am 27. Februar 2016.

| Daten zur Aufnahme | |
| - | - |
| Kamera | ASI120MC |
| Aufnahmeort | Bad Kreuznach |
| Teleskop | Celestron 8 Nexstar Evolution 3x Barlow |
| Nachbearbeitung | Registax 6, Lightroom |
| Kategorie | Sterne |



In [39]:
used_paths = []
for astropic in g("Astropic"):
    friendly_name = slugify(astropic["Title"], max_length=255, replacements=[
        ['Ä', 'AE'], ['ä', 'ae'],
        ['Ö', 'OE'], ['ö', 'oe'],
        ['Ü', 'UE'], ['ü', 'ue'],
        ['ß', 'ss'],
    ])
    
    year = astropic["Created"][:4]    
    folder = Path("..", "content", "galerie", year, friendly_name)
    
    index = 1    
    while str(folder) in used_paths:
        folder = Path("..", "content", "galerie", year, "{}-{}".format(friendly_name, index))
        index += 1
    
    used_paths.append(str(folder))
    download_link = get_download_link(astropic)
    file_extension = download_link.split(".")[-1]
    
    file_name = folder.name    
    folder.mkdir(exist_ok=True, parents=True)
    file_name = file_name + "." + file_extension
    
    target_file = folder.joinpath(file_name)
    
    download_file(download_link, target_file)
    
    content, yaml_dict = make_content(astropic)    
    yaml_dict["images"] = [target_file.name]
    write_file(folder, yaml_dict, content)  
    print("Wrote to", folder)

Wrote to ../content/galerie/2016/sternhimmel-bosenberg
Wrote to ../content/galerie/2016/big-moon-mosaic
Wrote to ../content/galerie/2016/tycho
Wrote to ../content/galerie/2016/sternhimmel-wald
Wrote to ../content/galerie/2016/flares-und-sonnenfleck
Wrote to ../content/galerie/2016/protuberanzen-12-mai-2016
Wrote to ../content/galerie/2016/merkur-transit-herschelprisma
Wrote to ../content/galerie/2016/protuberanzen
Wrote to ../content/galerie/2016/merkur-transit-zoom-h-alpha
Wrote to ../content/galerie/2016/beteigeuze
Wrote to ../content/galerie/2016/sirius
Wrote to ../content/galerie/2016/jupiter-io
Wrote to ../content/galerie/2016/m42-orionnebel
Wrote to ../content/galerie/2016/sternhimmel-muenchwald
Wrote to ../content/galerie/2016/sternhimmel-kuhberg
Wrote to ../content/galerie/2016/halbmond-1-shoot
Wrote to ../content/galerie/2016/halbmond
Wrote to ../content/galerie/2016/jupiter-gruen-kanal
Wrote to ../content/galerie/2016/la-perouse
Wrote to ../content/galerie/2016/langrenus-im-m